# 4. Таңбаланбаған деректермен жаттықтыру
# 4. Обучение на неразмеченных данных

In [3]:
from importlib.metadata import version

pkgs = ["matplotlib", 
        "numpy", 
        "tiktoken", 
        "torch",
        "tensorflow" # For OpenAI's pretrained weights
       ]
for p in pkgs:
    print(f"{p} version: {version(p)}")

matplotlib version: 3.10.0
numpy version: 1.26.4
tiktoken version: 0.8.0
torch version: 2.5.1
tensorflow version: 2.20.0


- Бұл тарауда біз LLM-ді алдын ала оқыту үшін оқыту циклін және модель бағалаудың қарапайым түрін іске асырамыз.
- Осы тараудың соңында біз сондай-ақ OpenAI-дың ашық қолжетімді алдын ала оқытылған салмақтарын моделімізге жүктейміз.

- В этой главе мы реализуем цикл обучения и базовую оценки модели для предобучения LLM.
- В конце этой главы мы также загрузим в нашу модель общедоступные предварительно обученные веса от OpenAI.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/chapter-overview.webp" width=1000px>

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/mental-model--0.webp" width=1000px>

# 4.1 Мәтін  модельдерін бағалау
# 4.1 Оценка текстовых моделей

- Біз бұл бөлімде алдыңғы өткентарауда пайдаланылған  GPT моделін инициализациялау процесын қысқаша қайталаудан бастаймыз.
- Содан кейін біз LLM-дерге арналған бастапқы бағалау метрикаларын талқылаймыз.
- Соңында, осы бөлімде біз бұл бағалау метрикаларын жаттықтыру және валидация деректер жиынына қолданамыз.

- Мы начинаем этот раздел с краткого повторения инициализации модели GPT с использованием кода из предыдущей главы.
- Затем мы обсуждаем базовые метрики оценки для LLM.
- Наконец, в этом разделе мы применим эти метрики оценки к обучающему и валидационному наборам данных.

### 4.1.1 GPT арқылы текст генерациялау
### 4.1.1 Генерация текста с помощью GPT

- Біз алдыңғы тараудағы кодты қолданып, GPT моделін инициализациялаймыз
- Мы инициализируем модель GPT, используя код из предыдущей главы.

In [4]:
import torch
from previous_chapters import GPTModel

GPT_CONFIG_124M = {
    "vocab_size": 50257,   # Vocabulary size
    "context_length": 256, # Shortened context length (orig: 1024)
    "emb_dim": 768,        # Embedding dimension
    "n_heads": 12,         # Number of attention heads
    "n_layers": 12,        # Number of layers
    "drop_rate": 0.1,      # Dropout rate
    "qkv_bias": False      # Query-key-value bias
}

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval();  # Disable dropout during inference

1.  **Біз $0.1$ dropout қолданамыз, бірақ қазіргі уақытта LLM-дерді dropout қолданбай жаттыөтыру жиі кездеседі.**
2.  **Сондай-ақ, заманауи LLM-дер сұраныс (query), кілт (key) және мән (value) матрицалары үшін `nn.Linear` қабаттарында ығысу (bias) векторларын пайдаланбайды (бұрынғы GPT модельдерінен айырмашылығы), бұл `"qkv_bias": False` орнату арқылы жүзеге асырылады.**
3.  **Біз модельді оқыту үшін есептеу ресурстарының талаптарын азайту мақсатында контекст ұзындығын (`context_length`) бар болғаны $256$ токенге дейін қысқартамыз, ал бастапқы $124$ миллион параметрлі GPT-$2$ моделі $1024$ токенді пайдаланған.**
    - **Бұл ноутбуктарда кодты орындай алу үшін жасалған.**
    - **Алайда, `context_length`-ті $1024$ токенге дейін арттыра алуға болады (бұл ешқандай код өзгерістерін талап етпейді).**
    - **Біз сондай-ақ кейінірек алдын ала оқытылған салмақтардан $1024$ `context_length`-і бар модельді жүктейтін боламыз.**
***
1.  **Мы используем dropout $0.1$ , но в настоящее время довольно часто обучают LLM без dropout.**
2.  **Современные LLM также не используют векторы смещения (bias) в слоях `nn.Linear` для матриц запроса (query), ключа (key) и значения (value) (в отличие от ранних моделей GPT), что достигается установкой `"qkv_bias": False`.**
3.  **Мы уменьшаем длину контекста (`context_length`) всего до $256$ токенов, чтобы снизить требования к вычислительным ресурсам для обучения модели, тогда как оригинальная модель GPT-$2$ с $124$ миллионами параметров использовала $1024$ токена.**
    - **Это сделано для того, чтобы  выполнять примеры кода на своих ноутбуках.**
    - **Однако, можно увеличить `context_length` до $1024$ токенов (это не потребует никаких изменений в коде).**
    - **Мы также загрузим модель с `context_length` $1024$ позже из предварительно обученных весов.**

- Мәтін жасау үшін алдыңғы тараудағы `generate_text_simple` функциясын қолданамыз.
- Бұған қоса, біз осы тарау бойында қолданылатын токен және мәтін ұсынымдары арасында түрлендіру үшін екі ыңғайлы функцияны (`text_to_token_ids` және `token_ids_to_text`) қолданамыз.

- Далее мы используем функцию `generate_text_simple` из предыдущей главы для генерации текста.
- Кроме того, мы используем две вспомогательные (или удобные) функции (`text_to_token_ids` и `token_ids_to_text`) для преобразования между токенами и текстовыми представлениями, которые мы используем на протяжении всей этой главы.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/gpt-process.webp" width=1000px>

In [5]:
import tiktoken
from previous_chapters import generate_text_simple

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you rentingetic wasnم refres RexMeCHicular stren


* Жоғарыда көріп тұрғанымыздай, модель әлі жаттықпағандықтан жақсы мәтін шығара алмайды.
* Келесі бөлімде модельдің шығарған нәтижелеріне арналған шығын (loss) метрикасын есептеу тәсілдерімен танысамыз, бұл жаттығу барысын өлшеуге мүмкіндік береді.
* Ал LLM модельдерін дәл баптау (finetuning) туралы келесі тарауларда модель сапасын бағалаудың қосымша әдістері енгізіледі.
---
* Как мы видим выше, модель не генерирует хороший текст, потому что она ещё не обучена.
* В следующем подразделе будут представлены метрики для вычисления функции потерь (loss) для сгенерированных выходных данных, которые можно использовать для оценки прогресса обучения.
* В следующих главах, посвящённых дообучению (finetuning) LLM, также будут рассмотрены дополнительные способы измерения качества модели.


### 4.1.2 Мәтін генерациясының шығынын (қатесін) есептеу: кросс-энтропия және perplexity (шатасу)
### 4.1.2 Вычисление функции потерь при генерации текста: кросс-энтропия и perplexity (затруднение)

* Айталық, бізде 2 жаттығу мысалына (қатарға) арналған токен ID-лерін қамтитын `inputs` тензоры бар.
* `inputs` мәндеріне сәйкес, `targets` тензоры модельдің генерациялауын қалаған токен ID-лерін қамтиды.
* Назар аударыңыз, `targets` — бұл `inputs` тензорының 1 позицияға ығыстырылған нұсқасы (2-тарауда деректер жүктеушіні жүзеге асырғанда түсіндірілгендей).
---
* Предположим, у нас есть тензор `inputs`, содержащий идентификаторы токенов для двух обучающих примеров (строк).
* Соответственно `inputs`, тензор `targets` содержит токен-идентификаторы, которые модель должна сгенерировать.
* Обратите внимание, что `targets` — это версия `inputs`, сдвинутая на одну позицию, как объяснялось в главе 2 при реализации загрузчика данных.


In [6]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
                       [40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
                        [1107,  588, 11311]]) #  " really like chocolate"]

* `inputs` тензорын модельге берген кезде, әрқайсысында 3 токен бары 2 кіріс мысалы үшін логиттер векторын аламыз.
* Әрбір токен сөздік өлшеміне сәйкес келетін 50,257 өлшемді вектор болып табылады.
* `Softmax` функциясын қолдану арқылы логиттер тензорын сол өлшемдегі ықтималдық мәндерін қамтитын тензорға айналдыра аламыз.
---
* Передавая тензор `inputs` в модель, мы получаем вектор логитов для двух входных примеров, каждый из которых состоит из трёх токенов.
* Каждый токен представляет собой вектор размерностью 50 257, что соответствует размеру словаря.
* Применяя функцию `softmax`, можно преобразовать тензор логитов в тензор той же размерности, содержащий вероятностные значения.


In [7]:
with torch.no_grad():
    logits = model(inputs)

probas = torch.softmax(logits, dim=-1) # Probability of each token in vocabulary
print(probas.shape) # Shape: (batch_size, num_tokens, vocab_size)

torch.Size([2, 3, 50257])


* Төмендегі суретте, мысал ретінде өте кішкентай сөздік қолданып, ықтималдық мәндерін қайтадан мәтінге түрлендіру процесі көрсетілген. Бұл тақырыпты біз алдыңғы тараудың соңында талқылағанбыз.
---
* На рисунке ниже, с использованием очень небольшого словаря для наглядности, показано, как мы преобразуем вероятностные значения обратно в текст — об этом мы говорили в конце предыдущей главы.


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/proba-to-text.webp" width=1000px>

* Алдыңғы тарауда талқыланғандай, ықтималдық мәндерін болжаған токен ID-леріне түрлендіру үшін `argmax` функциясын қолдануға болады.
* Жоғарыдағы softmax функциясы әр токен үшін 50,257 өлшемді вектор шығарды; ал `argmax` функциясы осы вектордағы ең жоғары ықтималдық мәнінің орнын қайтарады — бұл берілген токен үшін болжанған токен ID болып табылады.
---
* Как обсуждалось в предыдущей главе, мы можем применить функцию `argmax`, чтобы преобразовать вероятностные значения в предсказанные идентификаторы токенов.
* Функция softmax выше сгенерировала вектор размерностью 50 257 для каждого токена; функция `argmax` возвращает позицию с наибольшим значением вероятности в этом векторе — это и есть предсказанный идентификатор токена для данного токена.


* Бізде әрқайсысында 3 токен бар 2 кіріс партия (batch) болғандықтан, нәтижесінде 2×3 болжанған токен ID-лерін аламыз:
---
* Поскольку у нас есть 2 входных батча, каждый из которых содержит по 3 токена, мы получаем 2×3 предсказанных идентификатора токенов:

In [8]:
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

Token IDs:
 tensor([[[16657],
         [  339],
         [42826]],

        [[49906],
         [29669],
         [41751]]])


* Егер біз осы токендерді декодтасақ, олардың модельден күткен, яғни мақсатты токендерден айтарлықтай өзгеше екенін көреміз:
---
* Если декодировать эти токены, мы увидим, что они значительно отличаются от тех токенов, которые мы хотим, чтобы модель предсказала — то есть от целевых токенов:

In [9]:
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

Targets batch 1:  effort moves you
Outputs batch 1:  Armed heNetflix


* Бұл модель әлі жаттықпағандықтан болып тұр.
* Модельді жаттықтыру үшін оның дұрыс болжамдардан (мақсаттардан) қаншалықты алыс екенін білуіміз қажет.
---
* Это происходит потому, что модель ещё не была обучена.
* Чтобы обучить модель, нам нужно знать, насколько её предсказания отличаются от правильных (целевых) значений.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/proba-index.webp" width=1000px>

In [10]:
print(f"Probability shape: {probas.shape}")

Probability shape: torch.Size([2, 3, 50257])


* `2` - мәтін үлгілер саны   
* `3` - мәтіндегі сөздер саны   
* `50257` - сөздік өлшемі   
***
* `2` - количество образцов текста   
* `3` - количество слов в каждом тексте  
* `50257` - размерность словаря  

In [11]:
print(f"Correct target tokens  for first text: {targets[0].tolist()}")
print(f"Correct target tokens  for second text: {targets[1].tolist()}")

Correct target tokens  for first text: [3626, 6100, 345]
Correct target tokens  for second text: [1107, 588, 11311]


In [12]:
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], [3626, 6100, 345]] # "effort moves you"
print("Text 1:", target_probas_1)

text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], [1107,  588, 11311]] # "really like chocolate"
print("Text 2:", target_probas_2)

Text 1: tensor([7.4541e-05, 3.1061e-05, 1.1563e-05])
Text 2: tensor([1.0337e-05, 5.6776e-05, 4.7559e-06])


- Біз осы мәндердің барлығын барынша арттырып, оларды 1 ықтималдығына жақындатқымыз келеді.
- Математикалық оңтайландыруда **ықтималдық баллдардың** логарифмын максимизациялау ықтималдық баллдың өзін максимизациялауға қарағанда оңайырақ.
---
- Мы хотим максимизировать все эти значения, приближая их к вероятности, равной 1.
- В математической оптимизации максимизировать логарифм вероятности легче, чем саму вероятность.

In [13]:
# Compute logarithm of all token probabilities
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)

tensor([ -9.5042, -10.3796, -11.3677, -11.4798,  -9.7764, -12.2561])


- Сосын біз орташа логарифмдік ықтималдықты есептейміз
- Далее мы вычисляем средний логарифм вероятности

In [14]:
# Calculate the average probability for each token
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)

tensor(-10.7940)


* Басты мақсат – модель салмақтарын оңтайландыру арқылы осы орташа логарифмдік ықтималдықты мүмкіндігінше үлкен ету.
* Логарифмге байланысты, ең үлкен мүмкін мән $0$-ге тең, ал біз қазір $0$-ден алыспыз.
***
* Цель — сделать эту среднюю логарифмическую вероятность как можно большей (или: *максимизировать*) путем оптимизации весов модели.
* Из-за логарифма, максимально возможное значение равно $0$, и мы в настоящее время далеки от $0$.

* Терең оқытуда орташа логарифмдік ықтималдықты максимизациялаудың орнына, оның *теріс* мәнін минимизациялау қабылданған; біздің жағдайда $-10.7722$ мәнін $0$-ге жақындату үшін максимизациялаудың орнына, терең оқытуда біз $10.7722$ мәнін $0$-ге жақындату үшін минимизациялаймыз.
* $-10.7722$-ге қарама-қарсы мән, яғни $10.7722$, терең оқытуда **қиылыспалы энтропия шығыны** (cross-entropy loss) деп те аталады.
***
* В глубоком обучении вместо максимизации средней логарифмической вероятности принято минимизировать ее *отрицательное* значение; в нашем случае, вместо максимизации $-10.7722$, чтобы оно приближалось к $0$, в глубоком обучении мы бы минимизировали $10.7722$, чтобы оно приближалось к $0$.
* Значение, противоположное $-10.7722$, то есть $10.7722$, в глубоком обучении также называется **потерей перекрестной энтропии** (cross-entropy loss).

In [15]:
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)

tensor(10.7940)


* PyTorch-та алдыңғы қадамдарды орындайтын `cross_entropy` функциясы жүзеге асырылған.
***
* В PyTorch уже реализована функция `cross_entropy`, которая выполняет предыдущие шаги.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/cross-entropy.webp?123" width=1000px>

In [16]:
# Logits have shape (batch_size, num_tokens, vocab_size)
print("Logits shape:", logits.shape)

# Targets have shape (batch_size, num_tokens)
print("Targets shape:", targets.shape)

Logits shape: torch.Size([2, 3, 50257])
Targets shape: torch.Size([2, 3])


* PyTorch-тағы `cross_entropy` функциясы үшін біз бұл тензорларды **батч өлшемі (batch dimension) бойынша біріктіру арқылы** **жазықтауымыз** (тегістеуіміз) керек:
***
* Для функции `cross_entropy` в PyTorch мы хотим **'выпрямить' (сгладить)** эти тензоры, объединив их по **размерности батча** (batch dimension):

In [17]:
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()

print("Flattened logits:", logits_flat.shape)
print("Flattened targets:", targets_flat.shape)

Flattened logits: torch.Size([6, 50257])
Flattened targets: torch.Size([6])


* `Target` — бұл токен идентификаторлары, олар сондай-ақ логиттердегі максимизациялағымыз (1-ге) келетін индекстік позицияларды білдіреді.
* PyTorch-тағы `cross_entropy` функциясы softmax және логарифмдік ықтималдықты есептеу процестерін автоматты түрде орындайды, яғни максимизациялануы тиіс логиттердегі токен индекстері бойынша ішкі есептеулерді өзі атқарады.
---
* `Target` — это идентификаторы токенов, которые также представляют индексные позиции в логитах, которые мы хотим максимизировать.
* Функция `cross_entropy` в PyTorch автоматически выполняет применение softmax и вычисление логарифмической вероятности для тех индексов токенов в логитах, которые необходимо максимизировать.


In [18]:
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(loss)

tensor(10.7940)


- Perplexity (шатасу) - LLM-нің  cross-entropy шығынының дәрежелік көрсеткіші
- Perplexity (заблуждение) - это просто экспонента от потерь по перекрёстной энтропии

In [19]:
perplexity = torch.exp(loss)
print(perplexity)

tensor(48725.8203)


* Perplexity көбіне түсінуге жеңіл көрсеткіш ретінде қарастырылады, себебі оны модель әр қадамда сенімсіз болатын «сөздер көлемі» ретінде түсіндіруге болады (жоғарыдағы мысалда бұл 48 725 сөз немесе токен).
* Басқаша айтқанда, perplexity модель болжаған ықтималдықтар таралуы  нақты сөздердің таралуына қаншалықты жақын екенін көрсететін өлшем.
* Шығын (loss) сияқты, perplexity неғұрлым төмен болса — модельдің болжамдары нақты таралуға соғұрлым жақын дегенді білдіреді.
---
* Perplexity часто считается более интерпретируемым показателем, потому что её можно понимать как эффективный размер словаря, в котором модель испытывает неопределённость на каждом шаге (в приведённом выше примере это 48 725 слов или токенов).
* Иными словами, перплексия показывает, насколько хорошо распределение вероятностей, предсказанное моделью, соответствует реальному распределению слов в датасете.
* Подобно функции потерь, более низкое значение перплексии означает, что предсказания модели ближе к реальному распределению.


### Қарапайым мысал

Модель келесі сөйлемді жалғастыруға тырысып жатыр делік:

> “I love eating  ...”

Мүмкін келесі сөздер:
`["apples", "cars", "books", "tables"]`

* Егер модель **"apples"** сөзін **80%** ықтималдықпен дұрыс деп санаса —
  → модель **сенімді** → **төмен perplexity**

* Егер барлық 4 сөзге **25%** ықтималдық берсе —
  → модель **шатасқан** → **жоғары perplexity**
---
### Мағынасы

**Perplexity** шамамен былай түсіндіріледі:

> “Модель қанша нұсқаны бірдей дұрыс деп шатастырып тұр.”

Сондықтан:

* Perplexity ≈ 1 → модель өте сенімді (бір ғана дұрыс нұсқа бар).
* Perplexity ≈ 10 → модель 10 нұсқаның бірін таңдауға қиналып тұр.
---
### Қысқаша:

* **Cross-entropy loss** → Модель қаншалықты қателескенін көрсетеді.
* **Perplexity** → Модель қанша нұсқаның арасынан таңдай алмай тұрғанын көрсетеді.
* **Төмен perplexity** → жақсы модель деген сөз.
---
### Простой пример

Модель пытается продолжить предложение:

> “I love eating  ...”

Возможные слова:
`["apples", "cars", "books", "tables"]`

* Если модель считает **"apples"** правильным с вероятностью **80%** —
  → модель **уверена** → **низкая перплексия**

* Если она даёт всем по **25%** —
  → модель **в замешательстве** → **высокая перплексия**
---
### Смысл

**Перплексия** примерно означает:

> “Сколько вариантов модель считает одинаково возможными.”

Поэтому:

* Perplexity ≈ 1 → модель очень уверена (только один вариант).
* Perplexity ≈ 10 → модель запуталась между 10 вариантами.
---
### 🧾 Коротко:

* **Cross-entropy loss** → показывает, насколько модель ошибается.
* **Perplexity** → показывает, насколько модель запутана при выборе слова.
* **Меньше perplexity** → значит, модель работает лучше.


### 4.1.3 Жаттықтыру және валидация жиындарының шығындарын есептеу
### 4.1.3 Расчет потерь на обучающем и валидационном наборах


- Біз LLM-ді оқыту үшін салыстырмалы түрде шағын деректер жинағын пайдаланамыз (іс жүзінде, тек бір шығарма).
* Себептері:
    - Сіз код мысалдарын қолайлы GPU-сыз ноутбук компьютерде бірнеше минут ішінде іске қоса аласыз
    - Оқыту салыстырмалы түрде тез аяқталады (апталар емес, минуттар), бұл білім беру мақсаттары үшін жақсы
    - Біз қоғамдық қолданыстағы мәтінді пайдаланамыз, оны пайдалану құқықтарын бұзбай  қолдануға болады

- Мысалы, Llama 2 7B моделін 2 триллион токенде жаттықтыру үшін A100 GPU-ларда 184 320 GPU сағаты қажет болды

- Осы уақытта AWS-тегі 8xA100 бұлттық серверінің сағаттық құны шамамен 30 долларды құрайды
- Сонымен, шамамен есептеу бойынша, осы LLM-ді оқыту құны $184 320 / 8 \times 30$ = $690 000$ доллар болар еді.
---
- Мы используем относительно небольшой набор данных для обучения LLL (фактически, только одно произведение).
* Причины:
    - Вы можете запустить примеры кода за несколько минут на ноутбуке без подходящего GPU
    - Обучение завершается относительно быстро (минуты вместо недель), что хорошо для образовательных целей
    - Мы используем текст из общественного достояния, который можно включить в этот проект без нарушения каких-либо прав использования.
- Например, для обучения Llama 2 7B на 2 триллионах токенов потребовалось 184 320 GPU-часов на графических процессорах A100.
- На момент написания статьи почасовая стоимость облачного сервера 8xA100 на AWS составляет примерно 30 долларов США
- Таким образом, по приблизительным расчетам, обучение этой БЯМ стоило бы $184 320 / 8 \times 30$ = $690 000$ долларов США.

In [20]:
import os
import urllib.request

file_path = "time_machine.txt"
url = ("https://raw.githubusercontent.com/Azamat0315277/LLM_from_scratch/refs/heads/main/ch01/time_machine.txt")

if not os.path.exists(file_path):
    with urllib.request.urlopen(url) as response:
        text_data = response.read().decode('utf-8')
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text_data)
else:
    with open(file_path, "r", encoding="utf-8") as file:
        text_data = file.read()

- Мәтіннің дұрыс жүктелгенін тексеру үшін алғашқы және соңғы 99 таңбаны шығару
- Быстрая проверка того, что текст загружен корректно, путем вывода первых и последних 99 символов

In [21]:
# First 99 characters
print(text_data[:99])

 I.
 Introduction


The Time Traveller (for so it will be convenient to speak of him) was
expoundin


In [22]:
# Last 99 characters
print(text_data[-99:])

help produce our new eBooks, and how to
subscribe to our email newsletter to hear about new eBooks.


In [23]:
total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))

print("Characters:", total_characters)
print("Tokens:", total_tokens)

Characters: 197730
Tokens: 49509


- 49509 токен саны LLM-ді үйрету үшін өте қысқа, бірақ бұл уйрену мақсатында (біз кейінірек алдын ала үйретілген салмақтарды да жүктейміз)
- С 5,145 токенами текст очень короткий для обучения LLM, но это в образовательных целях (мы также загрузим предобученные веса позже)

- Келесі болып, біз деректер жиынын жаттықтыру және валидация жиынына бөлеміз және LLM жаттықтыру үшін пакеттерді дайындау мақсатында деректер жүктеушілерін қолданамыз.
- Визуализациялау мақсатында төмендегі сурет `max_length=6` деп қабылданады, бірақ оқыту жүктеушісі үшін біз `max_length`‑ты LLM‑нің қолдайтын контекст ұзындығына тең етіп орнатамыз.
- Төмендегі сурет тек кіріс токендерін қарапайымдылық үшін көрсетеді.
- Біз LLM‑ді мәтіндегі келесі сөзді болжауға үйрететіндіктен, мақсаттар (targets) осы кірістерге ұқсас, бірақ бір позицияға жылжытылған.
***
- Далее мы делим набор данных на обучающий и валидационный наборы и используем загрузчики данных для подготовки пакетов для обучения LLM.
- Для целей визуализации на рисунке ниже предполагается `max_length=6`, но для загрузчика обучения мы устанавливаем `max_length`, равный длине контекста, поддерживаемой LLM.
- На рисунке ниже показаны только входные токены для упрощения.
- Поскольку мы обучаем LLM предсказывать следующее слово в тексте, целевые значения (targets) выглядят так же, как и входные токены, за исключением того, что они сдвинуты на одну позицию.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/batching.webp" width=1000px>

In [ ]:
from previous_chapters import create_dataloader_v1
# Train/validation ratio
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]


torch.manual_seed(123)

train_loader = create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [25]:
# Sanity check

if total_tokens * (train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("Not enough tokens for the training loader. "
          "Try to lower the `GPT_CONFIG_124M['context_length']` or "
          "increase the `training_ratio`")

if total_tokens * (1-train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("Not enough tokens for the validation loader. "
          "Try to lower the `GPT_CONFIG_124M['context_length']` or "
          "decrease the `training_ratio`")

- Біз есептеу ресурстарына деген сұранысты азайту үшін және деректер жиынының өзі кішкентай болғандықтан салыстырмалы түрде кішкентай батч өлшемін қолданамыз.
- Мысалы, Llama 2 7B батч өлшемі 1024‑пен оқытылды.
***
- Мы используем относительно небольшой размер батчa, чтобы снизить требования к вычислительным ресурсам, а также потому, что сам набор данных изначально очень мал.
- Например, Llama 2 7B была обучена с размером батчa 1024.

* Деректердің дұрыс жүктелгенін тексеруге арналған қосымша тексеріс (міндетті емес):
* Дополнительная (необязательная) проверка того, что данные были загружены корректно:

In [26]:
print("Train loader:")
for x, y in train_loader:
    print(x.shape, y.shape)

print("\nValidation loader:")
for x, y in val_loader:
    print(x.shape, y.shape)

Train loader:
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256])

* Тағы бір қосымша тексеріс — токен өлшемдерінің күтілетін диапазонда екенін тексеру:
* Ещё одна дополнительная проверка — убедиться, что размеры токенов находятся в ожидаемом диапазоне:


In [27]:
train_tokens = 0
for input_batch, target_batch in train_loader:
    train_tokens += input_batch.numel()

val_tokens = 0
for input_batch, target_batch in val_loader:
    val_tokens += input_batch.numel()

print("Training tokens:", train_tokens)
print("Validation tokens:", val_tokens)
print("All tokens:", train_tokens + val_tokens)

Training tokens: 44544
Validation tokens: 4608
All tokens: 49152


* Келесі қадамда берілген батч үшін cross-entropy шығынын (loss) есептейтін көмекші функцияны іске асырамыз.
* Сонымен қатар, деректер жүктеушісінде (data loader)  белгіленген батчтар саны бойынша шығынды есептейтін екінші көмекші функцияны құрамыз.
---
* Далее мы реализуем вспомогательную функцию для вычисления cross-entropy потерь (loss) для заданного батча.
* Кроме того, мы реализуем вторую вспомогательную функцию, которая вычисляет потери для указанного количества батчей в загрузчике данных (data loader).

In [28]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

* Егер сізде CUDA қолдайтын GPU бар компьютер болса, кодқа ешқандай өзгеріс енгізбей-ақ LLM модель GPU-де оқытылады.
* `device` параметрі арқылы деректердің LLM моделінің орналасқан құрылғысымен бір құрылғыда жүктелуін қамтамасыз етеміз.
---
* Если у вас есть GPU с поддержкой CUDA, LLM будет обучаться на GPU без каких-либо изменений в коде.
* С помощью параметра `device` мы гарантируем, что данные загружаются на то же устройство, где находится модель LLM.


In [29]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Note:
# Uncommenting the following lines will allow the code to run on Apple Silicon chips, if applicable,
# which is approximately 2x faster than on an Apple CPU (as measured on an M3 MacBook Air).
# However, the resulting loss values may be slightly different.

if torch.cuda.is_available():
   device = torch.device("cuda")
elif torch.backends.mps.is_available():
   device = torch.device("mps")
else:
   device = torch.device("cpu")

print(f"Using {device} device.")


model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes


torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(val_loader, model, device)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Using mps device.
Training loss: 10.984251044262415
Validation loss: 10.96391593085395


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/mental-model-1.webp" width=1000px>

## 4.2 LLM жаттықтыру
## 4.2 Обучение LLM


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/train-steps.webp" width=1000px>

In [30]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            tokens_seen += input_batch.numel()
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )

    return train_losses, val_losses, track_tokens_seen


def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss


def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
    decoded_text = token_ids_to_text(token_ids, tokenizer)
    print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()

* Енді  оқыту функциясын пайдаланып, LLM моделін жаттыөтырайық:
* Теперь обучим LLM, используя определённую функцию обучения:

In [ ]:
# Note:
# Uncomment the following code to calculate the execution time
# import time
# start_time = time.time()

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="The Medical Man got", tokenizer=tokenizer
)

# Note:
# Uncomment the following code to show the execution time
# end_time = time.time()
# execution_time_minutes = (end_time - start_time) / 60
# print(f"Training completed in {execution_time_minutes:.2f} minutes.")

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator


def plot_losses(epochs_seen, tokens_seen, train_losses, val_losses):
    fig, ax1 = plt.subplots(figsize=(5, 3))

    # Plot training and validation loss against epochs
    ax1.plot(epochs_seen, train_losses, label="Training loss")
    ax1.plot(epochs_seen, val_losses, linestyle="-.", label="Validation loss")
    ax1.set_xlabel("Epochs")
    ax1.set_ylabel("Loss")
    ax1.legend(loc="upper right")
    ax1.xaxis.set_major_locator(MaxNLocator(integer=True))  # only show integer labels on x-axis

    # Create a second x-axis for tokens seen
    ax2 = ax1.twiny()  # Create a second x-axis that shares the same y-axis
    ax2.plot(tokens_seen, train_losses, alpha=0)  # Invisible plot for aligning ticks
    ax2.set_xlabel("Tokens seen")

    fig.tight_layout()  # Adjust layout to make room
    plt.savefig("loss-plot.pdf")
    plt.show()

epochs_tensor = torch.linspace(0, num_epochs, len(train_losses))
plot_losses(epochs_tensor, tokens_seen, train_losses, val_losses)

* Жоғарыдағы нәтижелерге қарап, модель бастапқыда түсініксіз сөз тізбектерін тудыратынын, ал соңына қарай грамматикалық тұрғыдан дұрыс сөйлемдер шығара алатынын көреміз.
* Алайда, тренинг және валидация жиынтықтарындағы шығын мәндеріне сүйенсек, модель шамадан тыс үйрене бастағанын байқаймыз.
* Егер соңында ол жазған бірнеше үзіндіні тексерсек, олардың тренинг жиынтығында дәлме-дәл бар екенін табамыз — яғни ол деректерді жаттап алған.
* Кейін біз бұл жаттап алуды белгілі бір деңгейде азайтатын декодтау стратегияларын қарастырамыз.
* Мұндағы артық үйрену себебі — тренинг жиынтығының өте кішкентай болуы және модельдің оны тым көп рет қайталап оқуы.
  * Мұндағы LLM тренингі негізінен оқу мақсатына арналған; бастысы — модельдің байланысты мәтін тудыра алатынын көру.
  * Бұл модельді қымбат жабдықтарда апталап немесе айлап жаттықтырудың орнына, біз кейінірек алдын ала жаттықтырылған салмақтарды жүктейміз.
---
* Глядя на результаты выше, мы видим, что модель изначально генерирует бессмысленные последовательности слов, тогда как ближе к концу она начинает создавать грамматически более или менее правильные предложения.
* Однако, по значениям потерь на обучающем и валидационном наборах видно, что модель начинает переобучаться.
* Если проверить несколько отрывков, написанных ею к концу обучения, можно заметить, что они буквально содержатся в обучающем наборе — модель просто запоминает данные.
* Позже мы рассмотрим стратегии декодирования, которые могут в некоторой степени уменьшить это запоминание.
* Обратите внимание, что переобучение здесь происходит из-за очень маленького обучающего набора и слишком большого количества итераций.
  * Обучение LLM здесь служит в основном образовательным целям; нам важно показать, что модель способна генерировать связный текст.
  * Вместо того чтобы тратить недели или месяцы на обучение этой модели на огромных объемах данных и дорогом оборудовании, мы позже загрузим предварительно обученные веса.


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/mental-model-2.webp" width=1000px>

## 4.3 Кездейсоқтықты басқаруға арналған декодтау стратегиялары
## 4.3 Стратегии декодирования для управления случайностью


* Генерация (inference) процесі жоғарыда үйретілген GPT сияқты шағын LLM үшін арзан, сондықтан оны орындау үшін GPU қолданудың қажеті жоқ.
* Бұрынғы бөлімде пайдаланған қарапайым оқу функциясының ішінде қолданылған `generate_text_simple` функциясын пайдалана отырып, біз жаңа мәтінді бір сөзден (немесе токеннен) біртіндеп тудыруымызға болады.
* Генерацияланған токен — бұл сөздік қорындағы барлық токендердің ішінде ең жоғары ықтималдық мәніне сәйкес келетін токен.
***
* Генерация (inference) недорог для небольшой LLM, как обученная выше модель GPT, поэтому нет необходимости использовать GPU для вывода, даже если вы использовали его для обучения.
* С помощью функции `generate_text_simple`  можно генерировать новый текст по одному слову (или токену) за раз.
* Следующий сгенерированный токен — это токен, соответствующий наибольшему вероятностному значению среди всех токенов в словаре.


In [ ]:
model.to("cpu")
model.eval()

tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=25,
    context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

* Жоғарыдағы `generate_text_simple` функциясын бірнеше рет орындасақ та, LLM әрдайым бірдей нәтиже шығарады.
* Енді біз екі *декодтау стратегияларын* — енгіземіз: *температуралық масштабтау (temperature scaling)* және *top-k* үлгілеу (sampling).
* Бұл тәсілдер модельге жасалған мәтіннің кездейсоқтығы мен әртүрлілігін басқаруға мүмкіндік береді.
***
* Даже если мы выполним функцию `generate_text_simple` несколько раз, LLM всегда будет генерировать одинаковый результат.
* Теперь мы введём две *стратегии декодирования*: *масштабирование температуры (temperature scaling)* и *top-k* выборку (sampling).
* Эти методы позволят модели управлять случайностью и разнообразием создаваемого текста.


### 4.3.1 Температуралық масштабтау
### 4.3.1 Масштабирование температуры

* Бұған дейін біз келесі токен ретінде ең жоғары ықтималдығы бар токенді таңдап алатынбыз (`torch.argmax` пайдаланып).
* Өзгеше  токен таңдап алу үшін, келесі токенді ықтималдықтар таралуынан кездейсоқ таңдау арқылы алуға болады — яғни `torch.multinomial(probs, num_samples=1)` функциясын қолданамыз.
* Мұнда әрбір индекстің таңдалу мүмкіндігі оның енгізу тензорындағы ықтималдығына сәйкес келеді.
***
* Ранее мы всегда выбирали токен с наибольшей вероятностью как следующий, используя `torch.argmax`.
* Чтобы добавить разнообразие, можно выбирать следующий токен из распределения вероятностей с помощью `torch.multinomial(probs, num_samples=1)`.
* Здесь вероятность выбора каждого индекса соответствует его вероятности во входном тензоре.

In [ ]:
import torch
vocab = { 
    "closer": 0,
    "every": 1, 
    "effort": 2, 
    "forward": 3,
    "inches": 4,
    "moves": 5, 
    "pizza": 6,
    "toward": 7,
    "you": 8,
} 

inverse_vocab = {v: k for k, v in vocab.items()}

# Suppose input is "every effort moves you", and the LLM
# returns the following logits for the next token:
next_token_logits = torch.tensor(
    [4.51, 0.89, -1.90, 6.75, 1.63, -1.62, -1.89, 6.28, 1.79]
)

probas = torch.softmax(next_token_logits, dim=0)
next_token_id = torch.argmax(probas).item()

# The next generated token is then as follows:
print(inverse_vocab[next_token_id])

In [ ]:
torch.manual_seed(123)
next_token_id = torch.multinomial(probas, num_samples=1).item()
print(inverse_vocab[next_token_id])

* `torch.argmax` арқылы анықтаудың орнына, біз `torch.multinomial(probas, num_samples=1)` функциясын қолданып, softmax таралуынан кездейсоқ таңдау арқылы  ең ықтимал токенді анықтаймыз.
* Мысал ретінде, бастапқы softmax ықтималдықтарын пайдаланып келесі токенді 1 000 рет таңдағанда не болатынын көрейік:
***
* Вместо того чтобы определять наиболее вероятный токен с помощью `torch.argmax`, мы используем `torch.multinomial(probas, num_samples=1)`, чтобы определить его, выбирая случайно из softmax-распределения.
* Для наглядности посмотрим, что произойдет, если выбрать следующий токен 1 000 раз, используя исходные softmax-вероятности:


In [ ]:
def print_sampled_tokens(probas):
    torch.manual_seed(123) # Manual seed for reproducibility
    sample = [torch.multinomial(probas, num_samples=1).item() for i in range(1_000)]
    sampled_ids = torch.bincount(torch.tensor(sample), minlength=len(probas))
    for i, freq in enumerate(sampled_ids):
        print(f"{freq} x {inverse_vocab[i]}")

print_sampled_tokens(probas)

* Біз таңдау процесін *temperature scaling* деп аталатын ұғым арқылы басқара аламыз.
* "Temperature scaling" — бұл жай ғана логиттерді 0-ден үлкен санға бөлу деген сөз.
* Температура мәні 1-ден үлкен болғанда, softmax қолданғаннан кейін токен ықтималдықтары біркелкілеу үлестіріледі.
* Ал температура 1-ден кіші болса, softmax қолданғаннан кейін үлестіру сенімдірек (өткір немесе шоғырланған) болады.
***
* Мы можем управлять распределением и процессом выбора с помощью концепции под названием *temperature scaling* (масштабирование температуры).
* "Temperature scaling" — это просто деление логитов на число больше нуля.
* Температура больше 1 приводит к более равномерному распределению вероятностей токенов после применения softmax.
* Температура меньше 1 делает распределение более уверенным (резким или пиковым) после применения softmax.

In [ ]:
def softmax_with_temperature(logits, temperature):
    scaled_logits = logits / temperature
    return torch.softmax(scaled_logits, dim=0)

# Temperature values
temperatures = [1, 0.1, 5]  # Original, higher confidence, and lower confidence

# Calculate scaled probabilities
scaled_probas = [softmax_with_temperature(next_token_logits, T) for T in temperatures]

In [ ]:
# Plotting
x = torch.arange(len(vocab))
bar_width = 0.15

fig, ax = plt.subplots(figsize=(5, 3))
for i, T in enumerate(temperatures):
    rects = ax.bar(x + i * bar_width, scaled_probas[i], bar_width, label=f'Temperature = {T}')

ax.set_ylabel('Probability')
ax.set_xticks(x)
ax.set_xticklabels(vocab.keys(), rotation=90)
ax.legend()

plt.tight_layout()
plt.savefig("temperature-plot.pdf")
plt.show()

* Temperature мәні 0.1 болғанда қайта масштабтау нәтижесінде үлестіру анағұрлым өткір болып, `torch.argmax` тәсіліне ұқсай бастайды — яғни ең ықтимал сөз дерлік әрқашан таңдалады:
***
* Мы можем заметить, что масштабирование с температурой 0.1 приводит к более резкому распределению, приближающемуся к `torch.argmax`, так что наиболее вероятное слово почти всегда выбирается:

In [ ]:
print_sampled_tokens(scaled_probas[1])

In [ ]:
print_sampled_tokens(scaled_probas[2])

### 4.3.2 Top-k іріктеу
### 4.3.2 Top-k выборка

* Шығыс құбылмалығын арттыру және мағынасыз сөйлемдердің ықтималдығын азайту үшін, біз таңдауды ең ықтимал топ-k токендермен шектеуге болады:
***
* Чтобы использовать более высокие температуры для увеличения разнообразия вывода и уменьшения вероятности бессмысленных предложений, можно ограничить выборку топ-k наиболее вероятными токенами:


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/topk.webp" width=1000px>

In [ ]:
top_k = 3
top_logits, top_pos = torch.topk(next_token_logits, top_k)

print("Top logits:", top_logits)
print("Top positions:", top_pos)

In [ ]:
new_logits = torch.where(
    condition=next_token_logits < top_logits[-1],
    input=torch.tensor(float("-inf")), 
    other=next_token_logits
)

print(new_logits)

In [ ]:
topk_probas = torch.softmax(new_logits, dim=0)
print(topk_probas)

### 4.3.3 Мәтін генерациялау функциясын өзгерту
### 4.3.3 Изменение функции генерации текста

* Алдыңғы екі бөлімде температура бойынша іріктеу және top-k іріктеу тәсілдері таныстырылды
* Енді осы екі ұғымды қолданып,  `generate_text_simple` функциясын өзгертіп, жаңа `generate` функциясын құрамыз:
***
* В двух предыдущих подразделах были представлены выборка по температуре и выборка top-k
* Теперь используем эти два понятия, чтобы изменить функцию `generate_text_simple`и  создать новую функцию `generate`:


In [40]:
def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):

    # For-loop is the same as before: Get logits, and only focus on last time step
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]

        # New: Filter logits with top_k sampling
        if top_k is not None:
            # Keep only top_k values
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(float("-inf")).to(logits.device), logits)

        # New: Apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

            # New (not in book): numerical stability tip to get equivalent results on mps device
            # subtract rowwise max before softmax
            logits = logits - logits.max(dim=-1, keepdim=True).values
            
            # Apply softmax to get probabilities
            probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (batch_size, 1)

        # Otherwise same as before: get idx of the vocab entry with the highest logits value
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch_size, 1)

        if idx_next == eos_id:  # Stop generating early if end-of-sequence token is encountered and eos_id is specified
            break

        # Same as before: append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1)

    return idx

In [ ]:
torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=15,
    context_size=GPT_CONFIG_124M["context_length"],
    top_k=25,
    temperature=1.4
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

## 4.4 PyTorch-та модель салмақтарын жүктеу және сақтау
## 4.4 Загрузка и сохранение весов модели в PyTorch


- Training LLMs is computationally expensive, so it's crucial to be able to save and load LLM weights

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/mental-model-3.webp" width=1000px>

* PyTorch-та  модель салмақтарын (`state_dict`) деп аталатын құрылымды `torch.save` функциясы арқылы сақтауға болады:
* В PyTorch рекомендуется сохранять веса модели, так называемый `state_dict`, применяя функцию `torch.save` к методу `.state_dict()`:


In [ ]:
torch.save(model.state_dict(), "model.pth")

* Содан кейін модельдің салмақтарын жаңа `GPTModel` данасына келесідей түрде жүктеуге болады:
* Затем мы можем загрузить веса модели в новый экземпляр модели `GPTModel` следующим образом:


In [ ]:
model = GPTModel(GPT_CONFIG_124M)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load("model.pth", map_location=device, weights_only=True))
model.eval();

* LLM модельдерін кәдімгі `SGD` оптимизаторы орнына `Adam` немесе `AdamW` сияқты  оптимизаторлармен жаттықтыру жиі қолданылады.
* Бұл оптимизаторлар әрбір модель салмағы үшін қосымша параметрлерді сақтайды, сондықтан егер болашақта алдын ала оқытуды жалғастыруды жоспарласақ, оларды да сақтау орынды:
***
* Часто LLM обучают с помощью оптимизаторов, таких как `Adam` или `AdamW`, вместо обычного `SGD`.
* Эти  оптимизаторы хранят дополнительные параметры для каждого веса модели, поэтому имеет смысл сохранить их, если мы планируем продолжить предварительное обучение позже:


In [ ]:
torch.save({
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    }, 
    "model_and_optimizer.pth"
)

In [ ]:
checkpoint = torch.load("model_and_optimizer.pth", weights_only=True)

model = GPTModel(GPT_CONFIG_124M)
model.load_state_dict(checkpoint["model_state_dict"])

optimizer = torch.optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.1)
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
model.train()

## 4.5 OpenAI-ден алдын ала үйретілген салмақтарды жүктеу
## 4.5 Загрузка предобученных весов из OpenAI


In [31]:
print("TensorFlow version:", version("tensorflow"))
print("tqdm version:", version("tqdm"))

TensorFlow version: 2.20.0
tqdm version: 4.67.1


In [ ]:
import os
import urllib.request
BASE_CONFIG = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "drop_rate": 0.0,       # Dropout rate
    "qkv_bias": True        # Query-key-value bias
}

file_name = "gpt2-small-124M.pth"
# file_name = "gpt2-medium-355M.pth"
# file_name = "gpt2-large-774M.pth"
# file_name = "gpt2-xl-1558M.pth"

url = f"https://huggingface.co/rasbt/gpt2-from-scratch-pytorch/resolve/main/{file_name}"

if not os.path.exists(file_name):
    urllib.request.urlretrieve(url, file_name)
    print(f"Downloaded to {file_name}")

Downloaded to gpt2-small-124M.pth


* Қосымша `"355M"`, `"774M"` және `"1558M"` мәндері де `model_size` аргументі ретінде қолдауға болады.
* Әр түрлі өлшемдегі бұл модельдердің айырмашылығы төмендегі суретте көрсетілген:
***
* В качестве альтернативы аргументы `model_size` также могут принимать значения `"355M"`, `"774M"` и `"1558M"`.
* Разница между моделями разных размеров показана на рисунке ниже:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch05_compressed/gpt-sizes.webp?timestamp=123" width=1000px>

* Жоғарыда біз 124M GPT-2 моделінің салмақтарын Python-ға жүктедік, бірақ оларды өзіміздің `GPTModel` үлгісіне әлі де орнатуымыз керек.
* Алдымен жаңа `GPTModel` данасын инициализациялаймыз.
* Негізгі GPT моделінде көпбасты зейін (multi-head attention) модуліндегі сұрау (query), кілт (key) және мән (value) матрицалары үшін сызықтық қабаттар (linear layers) ығысу векторларымен (bias vectors) инициализацияланған, бұл міндетті емес; алайда салмақтарды дұрыс жүктеу үшін біз де өзіміздің жүзеге асыруымызда `qkv_bias` параметрін `True` етіп орнатуымыз керек.
* Сондай-ақ, біз бастапқы GPT-2 модельдерінде қолданылған `1024` токендік контекст ұзындығын пайдаланамыз.
***
* Выше мы загрузили веса модели GPT-2 с 124 миллионами параметров в Python, однако нам всё ещё нужно перенести их в наш экземпляр `GPTModel`.
* Сначала мы инициализируем новый экземпляр `GPTModel`.
* В оригинальной модели GPT линейные слои для матриц запроса (query), ключа (key) и значения (value) в модуле многоголового внимания инициализировались с векторами смещения (bias), что не является обязательным или рекомендуемым; однако, чтобы корректно загрузить веса, нам также нужно включить это, установив параметр `qkv_bias` в значение `True` в нашей реализации.
* Мы также используем длину контекста `1024` токена, которая применялась в оригинальных моделях GPT-2.


In [ ]:
model_configs = {
    "gpt2-small (124M)": {"emb_dim": 768, "n_layers": 12, "n_heads": 12},
    "gpt2-medium (355M)": {"emb_dim": 1024, "n_layers": 24, "n_heads": 16},
    "gpt2-large (774M)": {"emb_dim": 1280, "n_layers": 36, "n_heads": 20},
    "gpt2-xl (1558M)": {"emb_dim": 1600, "n_layers": 48, "n_heads": 25},
}

# Copy the base configuration and update with specific model settings
model_name = "gpt2-small (124M)"  # Example model name
NEW_CONFIG = GPT_CONFIG_124M.copy()
NEW_CONFIG.update(model_configs[model_name])
NEW_CONFIG.update({"context_length": 1024, "qkv_bias": True})


gpt = GPTModel(NEW_CONFIG)
gpt.load_state_dict(torch.load(file_name, weights_only=True))
gpt.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
gpt.to(device)

* Егер модель дұрыс жүктелсе, оны бұрын жазған `generate` функциямыз арқылы жаңа мәтін генерациялау үшін пайдалануға болады:
***
* Если модель загружена корректно, мы можем использовать её для генерации нового текста с помощью нашей предыдущей функции `generate`:

In [54]:
torch.manual_seed(123)

token_ids = generate(
    model=gpt,
    idx=text_to_token_ids("Every effort moves you", tokenizer).to(device),
    max_new_tokens=25,
    context_size=NEW_CONFIG["context_length"],
    top_k=50,
    temperature=1
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you toward finding an ideal life. You don't have to accept your problems by trying to remedy them, because that would be foolish
